# Search Job Postings on LinkedIn with Exfunc

In this cookbook, we'll demonstrate how to search and filter job posting data from LinkedIn using Exfunc APIs

## Step 1: Set up the environment

First, let's install the required libraries and set up the Exfunc Python SDK and OpenAI client.

In [ ]:
%pip install exfunc openai

In [ ]:
from exfunc import Exfunc
from openai import OpenAI

sdk = Exfunc(
    api_key="YOUR_EXFUNC_API_KEY" # Get your API key from https://exfunc.com/
)
openai_client = OpenAI(
    api_key="YOUR_OPENAI_API_KEY"
)

## Step 2: Search LinkedIn job postings 

Next, we will search LinkedIn job postings with keywords and other criteria. This example will search for software engineer jobs in the US posted within last 24 hours. search-job-postings endpoint returns 10 results, so we will iterate through 5 pages to gather 50 results.

In [ ]:
job_postings = []
for i in range(1, 6):
    job_postings_result = sdk.linkedin.search_job_postings(request={
        "keywords": "software engineer", # Replace with any keywords you'd like to search
        "location": "US",
        "date_posted": "Past 24 hours",
        "page": i
    })
    job_postings.extend(job_postings_result.job_postings)

print(job_postings)

## Step 3: Enrich and filter job postings

Since we now have job posting urls from the search result, we can use it to enrich job posting details and filter based on criteria. For this particular example, we will filter based on job requirements mentioning AWS. Outputs the results as csv.

In [ ]:
import csv

def does_job_description_mention_aws(job_description):
    prompt = f"Return 'True' or 'False': please determine whether the job description requires AWS as a requirement.\n\nJob description:{job_description}"

    # OpenAI API call
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert at reviewing and analyzing job posting descriptions."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=10,
        temperature=0
    )

    # Extract sentiment from response
    mentions_aws = response.choices[0].message.content.strip()
    return mentions_aws

qualified_jobs = []
for job_posting in job_postings:
    get_job_posting_resp = sdk.linkedin.get_job_posting(request={
        "job_posting_url": job_posting.url
    })
    job_posting_detail = get_job_posting_resp.job_posting
    job_description = job_posting_detail.description
    mentions_aws = does_job_description_mention_aws(job_description)
    if 'true' in mentions_aws.lower():
        qualified_jobs.append(job_posting_detail)

with open('linkedin_search_results_filtered.csv', 'w',) as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['url', 'title', 'location', 'description', 'job_type', 'job_function', 'industries', 'seniority_level', 'applicants', 'date_posted', 'job_poster_name', 'job_poster_url', 'company_name', 'company_url'])
    for job_posting in qualified_jobs:
        writer.writerow([job_posting.url, job_posting.title, job_posting.location, job_posting.description, job_posting.job_type, job_posting.job_function, job_posting.industries, job_posting.seniority_level, job_posting.applicants, job_posting.date_posted, job_posting.job_poster_name, job_posting.job_poster_url, job_posting.company_name, job_posting.company_url])